<a href="https://colab.research.google.com/github/gtbnhyujmj/-Reference-BingX_Trades-Endpoints/blob/main/Test_Order_%5BFormal%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 測試下單
# 當前帳戶在指定的合約品種下單。（支持限價單、市價單、條件市價單等）

In [ ]:
# POST /openApi/swap/v2/trade/order/test

# 按 UID 限速：每秒 5 次
# 按 IP 組限速：
# API KEY 權限：永續合約交易

In [ ]:
# 導入標準函式庫
import time
import requests
import hmac
from hashlib import sha256

In [ ]:
# API 端點網址
APIURL = "https://open-api.bingx.com"
APIKEY = ""      # 輸入你的 API KEY
SECRETKEY = ""   # 輸入你的 SECRET KEY

In [ ]:
# 範例函式，送出測試下單請求
def demo():
    payload = {}   # POST 請求的資料主體，這裡為空
    path = '/openApi/swap/v2/trade/order/test'   # API 路徑
    method = "POST"   # HTTP 方法

    paramsMap = {    # 訂單參數
        "symbol": "BTC-USDT",      # 交易對
        "side": "BUY",             # 買進
        "positionSide": "LONG",    # 多單
        "type": "MARKET",          # 市價單
        "quantity": 5,             # 數量
        "takeProfit": "{\"type\": \"TAKE_PROFIT_MARKET\", \"stopPrice\": 31968.0,\"price\": 31968.0,\"workingType\":\"MARK_PRICE\"}"
        # 止盈條件，需用字串（符合API格式）
    }

    paramsStr = parseParam(paramsMap)   # 組合參數字串，含簽名
    return send_request(method, path, paramsStr, payload)   # 發送請求

In [ ]:
# 產生簽名
def get_sign(api_secret, payload):
    signature = hmac.new(api_secret.encode("utf-8"), payload.encode("utf-8"), digestmod=sha256).hexdigest()
    print("sign=" + signature)
    return signature

In [ ]:
# 發送 API 請求
def send_request(method, path, urlpa, payload):
    # 組合請求網址，包含簽名
    url = "%s%s?%s&signature=%s" % (APIURL, path, urlpa, get_sign(SECRETKEY, urlpa))
    print(url)

    headers = {
        'X-BX-APIKEY': APIKEY,    # 設定 API KEY 到標頭
    }

    response = requests.request(method, url, headers=headers, data=payload)  # 發送 HTTP 請求
    return response.text   # 回傳回應內容

In [ ]:
# 參數排序並加上時間戳
def parseParam(paramsMap):
    sortedKeys = sorted(paramsMap)   # 參數鍵排序
    paramsStr = "&".join(["%s=%s" % (x, paramsMap[x]) for x in sortedKeys])   # 組成參數字串

    if paramsStr != "":
        return paramsStr + "&timestamp=" + str(int(time.time() * 1000))      # 非空時，加上&timestamp
    else:
        return paramsStr + "timestamp=" + str(int(time.time() * 1000))       # 空參數時直接加timestamp

In [ ]:
# 主執行區，直接執行 demo 函式
if __name__ == '__main__':
    print("demo:", demo())